In [1]:
import fastf1
import pandas as pd
import numpy as np
from pymongo import MongoClient
import time
import warnings
import sys
import random
import os

# === [설정] ===
MONGO_URI = "mongodb://localhost:27017/"
DB_NAME = "f1_analytics_db"
FAILURE_LOG_FILE = "collection_failures.txt"

YEARS = range(2018, 2026)

SESSION_TYPES = {'Q': 'Qualifying', 'R': 'Race'}

# 경고 무시 및 캐시 설정
warnings.simplefilter(action='ignore', category=FutureWarning)
fastf1.Cache.enable_cache(r'D:\f1_cache_temp')

# === [MongoDB 연결] ===
try:
    client = MongoClient(MONGO_URI)
    db = client[DB_NAME]

    col_races = db["races_meta"]
    col_results = db["results"]
    col_laps = db["laps"]
    col_telemetry = db["telemetry"]

    # 인덱스 생성 (조회 속도 필수)
    col_telemetry.create_index([("Year", 1), ("Round", 1), ("SessionType", 1), ("Driver", 1)])
    print(f"✅ MongoDB Connected: {DB_NAME}")
except Exception as e:
    print(f"❌ MongoDB Connection Error: {e}")
    sys.exit(1)

# === [실패 기록 함수] ===
def log_failure(year, round_num, session_type, event_name, reason):
    try:
        timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
        log_msg = f"[{timestamp}] Year:{year} | Round:{round_num} | {session_type} | {event_name} | Reason: {reason}\n"
        with open(FAILURE_LOG_FILE, "a", encoding="utf-8") as f:
            f.write(log_msg)
    except Exception:
        pass

# === [저장 함수들] ===
def save_race_meta(session, year, round_num, session_type):
    try:
        if not hasattr(session, 'weather_data') or session.weather_data is None: return False
        weather = session.weather_data
        if weather.empty: return False

        avg_air_temp = weather['AirTemp'].mean()
        avg_track_temp = weather['TrackTemp'].mean()
        is_rain = weather['Rainfall'].any()

        unique_id = f"{year}_{round_num}_{session_type}"

        meta_doc = {
            "_id": unique_id,
            "Year": int(year),
            "Round": int(round_num),
            "SessionType": session_type,
            "Circuit": session.event.EventName,
            "Location": session.event.Location,
            "Country": session.event.Country,
            "Date": session.date,
            "TotalLaps": int(session.total_laps) if hasattr(session, 'total_laps') and pd.notna(session.total_laps) else None,
            "Avg_AirTemp": float(round(avg_air_temp, 1)) if pd.notna(avg_air_temp) else None,
            "Avg_TrackTemp": float(round(avg_track_temp, 1)) if pd.notna(avg_track_temp) else None,
            "IsRainy": bool(is_rain)
        }
        col_races.update_one({"_id": meta_doc["_id"]}, {"$set": meta_doc}, upsert=True)
        return True
    except Exception:
        return False

def save_results(session, year, round_num, session_type):
    try:
        if not hasattr(session, 'results') or session.results is None or session.results.empty: return
        results = session.results
        docs = []
        for _, row in results.iterrows():
            docs.append({
                "Year": int(year),
                "Round": int(round_num),
                "SessionType": session_type,
                "Driver": row['Abbreviation'],
                "DriverNumber": str(row['DriverNumber']),
                "Team": row['TeamName'],
                "Position": int(row['Position']) if pd.notna(row['Position']) else None,
                "GridPosition": int(row['GridPosition']) if pd.notna(row['GridPosition']) else 0,
                "Points": float(row['Points']) if pd.notna(row['Points']) else 0.0,
                "Status": row['Status'],
                "Q1": str(row['Q1']) if 'Q1' in row and pd.notna(row['Q1']) else None,
                "Q2": str(row['Q2']) if 'Q2' in row and pd.notna(row['Q2']) else None,
                "Q3": str(row['Q3']) if 'Q3' in row and pd.notna(row['Q3']) else None,
                "TimeDelta": str(row['Time']) if 'Time' in row and pd.notna(row['Time']) else None
            })
        if docs:
            col_results.delete_many({"Year": int(year), "Round": int(round_num), "SessionType": session_type})
            col_results.insert_many(docs)
    except Exception: pass

def save_laps(session, year, round_num, session_type):
    try:
        if not hasattr(session, 'laps') or session.laps is None or session.laps.empty: return 0
        laps = session.laps
        laps_docs = []
        for _, row in laps.iterrows():
            if pd.isna(row['LapTime']): continue
            laps_docs.append({
                "Year": int(year),
                "Round": int(round_num),
                "SessionType": session_type,
                "Driver": row['Driver'],
                "LapNumber": int(row['LapNumber']),
                "LapTime_Sec": row['LapTime'].total_seconds(),
                "Sector1_Sec": row['Sector1Time'].total_seconds() if pd.notna(row['Sector1Time']) else None,
                "Sector2_Sec": row['Sector2Time'].total_seconds() if pd.notna(row['Sector2Time']) else None,
                "Sector3_Sec": row['Sector3Time'].total_seconds() if pd.notna(row['Sector3Time']) else None,
                "Compound": row['Compound'],
                "TyreLife": int(row['TyreLife']) if pd.notna(row['TyreLife']) else None,
                "IsAccurate": bool(row['IsAccurate'])
            })
        if laps_docs:
            col_laps.delete_many({"Year": int(year), "Round": int(round_num), "SessionType": session_type})
            col_laps.insert_many(laps_docs)
            return len(laps_docs)
    except Exception: return 0
    return 0

def save_telemetry(session, year, round_num, session_type):
    try:
        if not hasattr(session, 'laps') or session.laps is None or session.laps.empty: return 0
        drivers = session.drivers
        telemetry_buffer = []
        print(f"      └── 📡 Telemetry Fetching ({len(drivers)} drivers)...")

        for driver in drivers:
            try:
                driver_laps = session.laps.pick_driver(driver)
                if driver_laps.empty: continue
                try: telemetry = driver_laps.get_telemetry()
                except Exception: continue

                cols = ['Date', 'SessionTime', 'Distance', 'Speed', 'RPM', 'Gear',
                        'Throttle', 'Brake', 'DRS', 'Steering', 'X', 'Y', 'Z']
                available_cols = [c for c in cols if c in telemetry.columns]
                records = telemetry[available_cols].copy()

                if 'SessionTime' in records.columns:
                    records['SessionTime'] = records['SessionTime'].dt.total_seconds()

                records['Year'] = int(year)
                records['Round'] = int(round_num)
                records['SessionType'] = session_type
                records['Driver'] = driver
                telemetry_buffer.extend(records.to_dict('records'))
            except Exception: continue

        if telemetry_buffer:
            col_telemetry.delete_many({"Year": int(year), "Round": int(round_num), "SessionType": session_type})
            batch_size = 50000
            for i in range(0, len(telemetry_buffer), batch_size):
                col_telemetry.insert_many(telemetry_buffer[i:i+batch_size])
        return len(telemetry_buffer)
    except Exception: return 0

# === [메인 실행 루프] ===
def main():
    print("🚀 F1 Deep Analysis Data Collector V2 (Smart Recovery Mode)")
    print("===========================================================")
    print(f"📅 Target Years: {list(YEARS)}")

    for year in YEARS:
        try:
            schedule = fastf1.get_event_schedule(year)
            # 현재 시점보다 과거에 있었던 이벤트만 필터링
            completed_races = schedule[schedule['EventDate'] < pd.Timestamp.now()]

            print(f"\n📅 Year: {year} - Found {len(completed_races)} Events (Checking for Testing...)")

            for _, row in completed_races.iterrows():
                round_num = row['RoundNumber']
                event_name = row['EventName']

                # [수정됨] Pre-Season (Round 0) 감지 시 즉시 스킵
                if round_num == 0:
                    print(f"   🚫 Skipped Pre-Season Testing: {event_name} (Round 0)")
                    continue

                for session_key, session_name in SESSION_TYPES.items():

                    unique_id = f"{year}_{round_num}_{session_name}"

                    # 메타데이터 존재 여부 확인
                    meta_exists = col_races.count_documents({"_id": unique_id}) > 0

                    # 텔레메트리 데이터가 충분히(100개 이상) 있는지 확인
                    telemetry_count = col_telemetry.count_documents({
                        "Year": int(year),
                        "Round": int(round_num),
                        "SessionType": session_name
                    })

                    # 메타도 있고 텔레메트리도 충분하면 스킵
                    if meta_exists and telemetry_count > 100:
                        print(f"   ⏩ Skipped: R{round_num} {session_name} (Complete: {telemetry_count} pts)")
                        continue

                    status_msg = "Retry (Partial Data)" if meta_exists else "New"
                    print(f"   📥 Collecting [{status_msg}]: R{round_num} [{session_name}] - {event_name} ...")

                    try:
                        session = fastf1.get_session(year, round_num, session_key)
                        session.load(telemetry=True, laps=True, weather=True)

                        save_race_meta(session, year, round_num, session_name)
                        save_results(session, year, round_num, session_name)
                        l_cnt = save_laps(session, year, round_num, session_name)
                        t_cnt = save_telemetry(session, year, round_num, session_name)

                        if l_cnt == 0 and t_cnt == 0:
                            print(f"      ⚠️ Warning: No data saved. Logging failure.")
                            log_failure(year, round_num, session_name, event_name, "Zero Data Saved")
                            print("      🛑 Potential issue. Sleeping for 30s...")
                            time.sleep(30)
                        else:
                            print(f"      ✅ Success! Laps: {l_cnt}, Tel: {t_cnt}")
                            sleep_time = random.uniform(5, 10)
                            print(f"      💤 Cooling down for {sleep_time:.1f}s...")
                            time.sleep(sleep_time)

                    except Exception as e:
                        print(f"      ❌ Critical Load Error ({session_name}): {e}")
                        log_failure(year, round_num, session_name, event_name, str(e))
                        print("      🛑 Critical error. Sleeping for 60s...")
                        time.sleep(60)

        except Exception as e:
            print(f"❌ Critical Year Error ({year}): {e}")
            time.sleep(60)

    print("\n✨ All collection tasks finished.")

if __name__ == "__main__":
    main()

✅ MongoDB Connected: f1_analytics_db
🚀 F1 Deep Analysis Data Collector V2 (Smart Recovery Mode)
📅 Target Years: [2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025]


core           INFO 	Loading data for Australian Grand Prix - Race [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info



📅 Year: 2018 - Found 21 Events (Checking for Testing...)
   ⏩ Skipped: R1 Qualifying (Complete: 439984 pts)
   📥 Collecting [Retry (Partial Data)]: R1 [Race] - Australian Grand Prix ...


req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
core        WARNING 	Fixed incorrect tyre stint information for driver '28'
req            INFO 	Using cached data for car_data
req            INFO 	No cached data found for position_data. Loading data...
_api           INFO 	Fetching position data...
core        WARNING 	Car position data is unavailable!
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core        WARNING 	Driver 5 completed the race distance 00:00.123000 before the recorded end of the session.
core           INFO 	Finished loading data for 20 drivers: ['5', '44', '7', '3', '14', '33', '27', '77', '2', '55

      └── 📡 Telemetry Fetching (20 drivers)...
      ✅ Success! Laps: 937, Tel: 0
      💤 Cooling down for 6.3s...


...

      └── 📡 Telemetry Fetching (20 drivers)...
      ✅ Success! Laps: 303, Tel: 459851
      💤 Cooling down for 5.6s...


core           INFO 	Loading data for Las Vegas Grand Prix - Race [v3.6.1]
req            INFO 	No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...


   📥 Collecting [New]: R22 [Race] - Las Vegas Grand Prix ...


req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
_api        WARNING 	Failed to align laps for driver

      └── 📡 Telemetry Fetching (20 drivers)...
      ✅ Success! Laps: 884, Tel: 662793
      💤 Cooling down for 8.5s...

✨ All collection tasks finished.
